In [2]:


import numpy as np
import recordlinkage as rl
from recordlinkage.base import BaseCompareFeature
import pandas as pd

class GeoTable(BaseCompareFeature):
    '''
    Designed for sex spectrum 1, 2, 9: 9 for unknown
    
    This class implements a comparison function for sex spectrum 1, 2, 9. It computes a vectorized output based on the input series s1 and s2.
    The conditions for the comparison are as follows:
    - If both s1 and s2 are equal to 2, the output is 1.
    - If both s1 and s2 are equal to 1, the output is 1.
    - If either s1 or s2 is equal to 9, the output is 0.5.
    - For all other cases, the output is 0.
    '''
    # def __init__(self, s1, s2):
    #     super(sex_1_2_9, self).__init__()
    
    def _compute_vectorized(self, parID_1, parID_2, pairs_within_10km, *args):
        
        if parID_1 == parID_2: return True
        # # parse "table" contained in args
        # pairs_within_10km = args[0]
        
        if parID_1 in pairs_within_10km.keys():
            return pairs_within_10km[parID_1] == parID_2
        else: return False
        pass
    
# use
# # read
import pickle
# name = ""
name = "61-51"
filename = f'../../Output/{name}_pool_idx.sav'
# pickle.dump(candidates, open(filename, 'wb'))


In [3]:
candidates = pickle.load(open(filename, 'rb'))[0:100]

df1 = pd.read_parquet("../../Census_samples/Whole_proc/Whole_1861_mom_pop_sp")
df2 = pd.read_parquet("../../Census_samples/Whole_proc/Whole_1851_mom_pop_sp")
# [candidates.get_level_values(0).unique()].reset_index()

In [4]:
# candidates.get_level_values(0)
df1 = df1[candidates.get_level_values(0).unique()].reset_index()

KeyError: "None of [Index([5, 4, 6, 10293], dtype='int64', name='recid_1')] are in the [columns]"

In [ ]:
# buffer_10k = pd.read_csv('/notebooks/data/Buffer_10k.csv')
buffer_10k = pd.read_csv('../../Census_samples/Buffer_10k.csv')
# par_coordinates = pd.read_csv('../Census_samples/Conparid_coordinates.csv')

pairs_within_10km = {pair['conparid_1']:pair['conparid_2'] for pair in buffer_10k.to_dict(orient = 'records')}

# from recordlinkage.index import SortedNeighbourhood
    #     indexer = SortedNeighbourhood(
    #     's', window=3
    #         )

index = rl.Index()
index.full()
candidates = index.index(df1, df2)

comparer = rl.Compare(GeoTable('parID', 'parID', pairs_within_10km, label = 'par_witin_10km'))
comparer = rl.Compare(GeoTable('birth_parID', 'birth_parID', pairs_within_10km, label = 'birth_par_witin_10km'))

compared = comparer.compute(candidates, df1, df2)


# result = result.drop(columns = ['parID_1', 'parID_2'])
# print(f"done with par_witin_10km")

# result = result.drop(columns = ['birth_parID_1', 'birth_parID_2'])
# result = result[result.birth_par_witin_10km == True]
# print(f"done with birth_par_witin_10km")
# result.to_parquet('../Output/61-51_pool_geo_big.parquet')

NameError: name 'pd' is not defined